In [1]:
import overpy
import pandas as pd
from unidecode import unidecode
import numpy as np
from locations_utils import update_locations_level, merge_nodes, delete_empty_mappings, set_missing_details
import sqlite3

In [2]:
osm_api = ""
loc_db = "../../data/locations.sqlite"

In [3]:
osm = overpy.Overpass(url=osm_api)

### Load continents

In [4]:
def query_continents(osm):
    query = """
        [out:json][timeout:3600][maxsize:1073741824];
        node["place"="continent"];
        out body;
    """
    return osm.query(query)

In [5]:
def read_continents_records(result, tags=[]):
    nodes = result.nodes
    lat = [float(x.lat) for x in nodes]
    lng = [float(x.lon) for x in nodes]
    df_dict = {'lng': lng, 'lat': lat}
    for tag in tags:
        df_dict[tag] = [d.tags.get(tag) for d in nodes]
    return pd.DataFrame(df_dict)

In [6]:
continents = read_continents_records(query_continents(osm), tags=['name:en'])
continents['continent_id'] = update_locations_level(loc_db, continents[['name:en']].rename(columns={'name:en': 'name'}), update_counts=False)
set_missing_details(loc_db, continents.rename(columns={'continent_id': 'id'}), tags=['lat', 'lng'])

Adding 1 new nodes to locations database


### Load countries

In [7]:
def query_countries(osm):
    query = """
        [out:json][timeout:3600][maxsize:1073741824];
        rel["ISO3166-1"]->.a;
        node(r.a:"label")->.b;
        node(r.a:"admin_centre")->.c;
        (.a;.b;.c;);
        out center;
    """
    return osm.query(query)

In [8]:
def read_countries_records(result, tags=[]):
    lat = []
    lng = []
    for d in result.relations:
        # Label as center
        center = next(filter(lambda x: x.role == "label", d.members), None)
        # If label is missing than admin_centre (capital)
        if center is None:
            center = next(filter(lambda x: x.role == "admin_centre", d.members), None)
        # If any of above is available than get referenced node
        if center is not None:
            center = center.resolve()
            lat.append(float(center.lat))
            lng.append(float(center.lon))
        # Else get caluclated centre from borders
        if center is None:
            lat.append(float(d.center_lat))
            lng.append(float(d.center_lon))
    # Dict of columns
    df_dict = {'lng': lng, 'lat': lat}
    for tag in tags:
        df_dict[tag] = [d.tags.get(tag) for d in result.relations]
    return pd.DataFrame(df_dict)


In [9]:
def fix_countries_dataframe(df, continents):
    have_eng_name = ~df['name:en'].isnull()
    # Replace default names with english names if available
    df.loc[have_eng_name, 'name'] = df[have_eng_name]['name:en']
    
    # Remove rows with land mass in name
    df = df[~df['name'].str.endswith('(land mass)')].copy()
    df = df.drop_duplicates('ISO3166-1')
    
    # List of names of correct continents
    available_continents = continents['name:en'].tolist()
    # Remove incorrect continent names
    df['is_in:continent'] = [x if x in available_continents else None for x in df['is_in:continent']]
    
    # Row with missing continent field
    without_continent = df['is_in:continent'].isnull()
    continent_assigned = []
    for index, row in df[without_continent].iterrows():
        # Get distance from each continent centre
        dists = []
        for index2, continent in continents.iterrows():
            dists.append((row['lat'] - continent['lat'])**2 + (row['lng'] - continent['lng'])**2)
        order = np.argsort(dists)
        # Set closest continent
        continent_assigned.append(continents.iloc[order[0]]['name:en'])
    df.loc[without_continent, 'is_in:continent'] = continent_assigned
    return df

In [10]:
countries_result = query_countries(osm)

In [11]:
countries_df = read_countries_records(countries_result, tags=['name', 'name:en', 'ISO3166-1', 'is_in:continent'])
countries_df = fix_countries_dataframe(countries_df, continents)
countries_df['continent_id'] = update_locations_level(loc_db, countries_df[['is_in:continent']].rename(columns={'is_in:continent': 'name'}), update_counts=False)
countries_df['country_id'] = update_locations_level(loc_db, countries_df[['continent_id', 'name']].rename(columns={'continent_id': 'parent_id'}), update_counts=False)
set_missing_details(loc_db, countries_df.rename(columns={'ISO3166-1': 'iso_code', 'country_id': 'id'}))

Adding 0 new nodes to locations database
Adding 110 new nodes to locations database


### Load states

In [12]:
def query_states(osm):
    query = """
        [out:json][timeout:3600][maxsize:1073741824];
        rel["ISO3166-2"]["boundary"="administrative"];
        out center;
    """
    return osm.query(query)

In [13]:
def read_states_records(result, tags=[]):
    rels = result.relations
    lat = [float(x.center_lat) for x in rels]
    lng = [float(x.center_lon) for x in rels]
    df_dict = {'lng': lng, 'lat': lat}
    for tag in tags:
        df_dict[tag] = [d.tags.get(tag) for d in rels]
    return pd.DataFrame(df_dict)

In [14]:
def fix_states_dataframe(df, countries):
    df['ISO3166-1'] = [x.split('-')[0] for x in df['ISO3166-2']]
    df = pd.merge(df, countries[['ISO3166-1', 'country_id']], on='ISO3166-1')
    return df

In [15]:
states_result = query_states(osm)

In [16]:
states_df = read_states_records(states_result, tags=['ISO3166-2', 'name', 'name:en'])
states_df = fix_states_dataframe(states_df, countries_df)

In [17]:
# Load mappings from default name
states_df['state_id'] = update_locations_level(loc_db, states_df[['country_id', 'name']].rename(columns={'country_id': 'parent_id'}), update_counts=False)
set_missing_details(loc_db, states_df.rename(columns={'ISO3166-2': 'iso_code', 'state_id': 'id'}))

Adding 4107 new nodes to locations database


In [18]:
# Load mappings from english name
states_with_eng_name = states_df[~states_df['name:en'].isnull()].copy()
states_with_eng_name['state_id_2'] = update_locations_level(loc_db, states_with_eng_name[['country_id', 'name:en']].rename(columns={'country_id': 'parent_id', 'name:en': 'name'}), update_counts=False)
set_missing_details(loc_db, states_with_eng_name.rename(columns={'ISO3166-2': 'iso_code', 'state_id_2': 'id'}))

Adding 1695 new nodes to locations database


In [19]:
# Merge mappings of default and english names
for state_id, rows in states_with_eng_name[['state_id', 'state_id_2']].groupby('state_id'):
    merge_nodes(loc_db, [state_id] + list(rows['state_id_2']))

In [20]:
# Delete unused nodes left after merging
delete_empty_mappings(loc_db)